## Overjeno Šifriranje

In [ ]:
import os
import time
import base64

from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import hashes, padding, hmac
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.fernet import Fernet

### Overjeno AES-GCM v Pythonu

In [ ]:
def enc_gcm(key: bytes , pt: bytes, nonce: bytes) -> bytes:
    aesgcm = AESGCM(key)
    ct = aesgcm.encrypt(nonce, pt, None)
    return ct


def dec_gcm(key: bytes, ct: bytes, nonce: bytes) -> bytes:
    aesgcm = AESGCM(key)
    pt = aesgcm.decrypt(nonce, ct, None)
    return pt

### Overjeno AES-GCM v Javi

```java
import fri.isp.Agent;
import fri.isp.Environment;
import java.nio.charset.StandardCharsets;
import java.security.Key;
import javax.crypto.Cipher;
import javax.crypto.KeyGenerator;
import javax.crypto.spec.GCMParameterSpec;
import java.util.HexFormat;
```

Uporabite AES-GCM in zavarujte komunikacijsko sejo, tako da bodo vsa sporočila šifrirana
```java
public class CommunicationExampleGCM {
    public static void main(String[] args) throws Exception {
        final Environment env = new Environment();  // Simulacijsko okolje, v katerem bivajo agenti

        final Key key = KeyGenerator.getInstance("AES").generateKey();  // Generiramo skupni skrivni ključ

        env.add(new Agent("ana") {
            @Override
            public void task() throws Exception {
                final byte[] pt = "Zdravo Bor, tukaj Ana.".getBytes(StandardCharsets.UTF_8);

                final Cipher ana = Cipher.getInstance("AES/GCM/NoPadding");
                ana.init(Cipher.ENCRYPT_MODE, key);
                final byte[] ct = ana.doFinal(pt);
                final byte[] iv = ana.getIV();

                send("bor", ct);
                send("bor", iv);
                print("Poslala sem sporočilo: %s", ct);
            }
        });

        env.add(new Agent("bor") {
            @Override
            public void task() throws Exception {
                final byte[] ct = receive("ana");
                final byte[] iv = receive("ana");

                final Cipher bor = Cipher.getInstance("AES/GCM/NoPadding");

                bor.init(Cipher.DECRYPT_MODE, key, new GCMParameterSpec(128, iv));
                final byte[] pt = bor.doFinal(ct);

                print("Sporočilo se glasi: '%s", new String(pt, StandardCharsets.UTF_8));
            }
        });
        env.connect("ana", "bor");  // Povežemo Ano in Bora
        env.start();    // zaženemo simulacijsko okolje
    }
}
```

#### Dodaten zgled: AES-GCM v Javi

```java
public class GCMExample {
    public static void main(String[] args) throws Exception {
        final String message = "Moje sporočilo.";    // Ustvarimo sporočilo
        final byte[] pt = message.getBytes(StandardCharsets.UTF_8);
        System.out.printf("Pošiljam: '%s'%n", new String(pt, StandardCharsets.UTF_8));

        final Key key = KeyGenerator.getInstance("AES").generateKey();  // Ustvarimo naključen ključ

        // Ana šifrira sporočilo
        final Cipher ana = Cipher.getInstance("AES/GCM/NoPadding"); // ustvarimo algoritem AES-GCM
        ana.init(Cipher.ENCRYPT_MODE, key); // pri tem klicu se IV samodejno izbere
        final byte[] ct = ana.doFinal(pt); // šifriramo
        final byte[] iv = ana.getIV(); // preberemo IV

        System.out.println("PT: " + HexFormat.of().formatHex(pt));  
        System.out.println("IV: " + HexFormat.of().formatHex(iv));
        System.out.println("CT: " + HexFormat.of().formatHex(ct));

        // Bor dešifrira sporočilo
        final Cipher bor = Cipher.getInstance("AES/GCM/NoPadding");
        bor.init(Cipher.DECRYPT_MODE, key, new GCMParameterSpec(128, iv));  // GCMParameterSpec(dolzina_značke, IV)
        final byte[] dt = bor.doFinal(ct); // dešifriramo
        
        System.out.println("PT: " + HexFormat.of().formatHex(dt)); // izpišemo dešifrirano vrednost
        System.out.printf("Prejel sem: '%s'%n", new String(dt, StandardCharsets.UTF_8));
    }
}
```

### Izpeljava ključa iz gesla z PBKDF2HMAC
[Documentation](https://cryptography.io/en/latest/hazmat/primitives/key-derivation-functions/#pbkdf2)

In [ ]:
def gen_key(password: str, salt: bytes, iterations: int=1000000) -> bytes:
    pbk = PBKDF2HMAC(hashes.SHA256(), 16, salt, iterations)     # iterations nam pove koliko krat naj se funkcija izvede (vsaj 1000000)
    return pbk.derive(password.encode())

gen_key("password1234", os.urandom(16))

```java
public class KeyDerivation {
    public static void main(String[] args) throws Exception {
        final String password = "hunter2";
        final byte[] salt = new byte[16];               // ustvarimo naključno sol
        // final byte[] salt = HexFormat.of().parseHex("e24701ca2d923d43e50a405ec718f3af");
        SecureRandom.getInstanceStrong().nextBytes(salt);
        
        // Uporabimo PBKDF2 z HMAC-SHA256, dodamo sol, povemo število iteracij in želeno dolžino ključa
        final SecretKeyFactory pbkdf = SecretKeyFactory.getInstance("PBKDF2WithHmacSHA256");
        final KeySpec specs = new PBEKeySpec(password.toCharArray(), salt, 1000000, 128);
        final Key key = pbkdf.generateSecret(specs);

        System.out.printf("key = %s%n", HexFormat.of().formatHex(key.getEncoded()));
        System.out.printf("len(key) = %d bytes%n", key.getEncoded().length);
    }
}
```

## Telefonski imenik

- prvih 16 bajtov je sol,
- drugih 16 bajtov je IV,
- preostali bajti predstavljajo tajnopis z značko overitvene kode sporočila.

In [ ]:
import json
from getpass import getpass
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import os

file_path = "pota_do_imenika/phone_book.enc"

SALT_SIZE = 16
IV_SIZE = 16
KEY_LEN = 32  # AES-256
PBKDF2_ITER = 200_000


def derive_key(password: str, salt: bytes) -> bytes:
    from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
    from cryptography.hazmat.primitives import hashes
    from cryptography.hazmat.backends import default_backend

    kdf = PBKDF2HMAC(
        algorithm=hashes.SHA256(),
        length=KEY_LEN,
        salt=salt,
        iterations=PBKDF2_ITER,
        backend=default_backend()
    )
    return kdf.derive(password.encode("utf8"))

def load_phone_book(password: str,file: str) -> dict[str, str]:
    try:
        with open(file, 'br') as h:
            data = h.read()
    except (FileNotFoundError, json.JSONDecodeError):
        print(f"Could not load {file}, creating an empty phone book.")
        return {}
    
    if len(data) < (SALT_SIZE + IV_SIZE + 1):
        print(f"File {file} is corrupted or too small, creating an empty phone book.")
        return {}

    salt = data[:SALT_SIZE]
    iv = data[SALT_SIZE:SALT_SIZE + IV_SIZE]
    ciphertext = data[SALT_SIZE + IV_SIZE:]
    
    try:
        key = derive_key(password, salt)
        aesgcm = AESGCM(key)
        pt = aesgcm.decrypt(iv, ciphertext, None)
        return json.loads(pt.decode("utf8"))
    except Exception as e:
        print(f"Decryption failed: {e}. Creating an empty phone book.")
        return {}
    
    
def save_phone_book(phone_book: dict[str, str], file: str, password: str) -> None:
    pt = json.dumps(phone_book).encode("utf8")
    
    salt = os.urandom(SALT_SIZE)
    iv = os.urandom(IV_SIZE)
    key = derive_key(password, salt)
    aesgcm = AESGCM(key)
    ct = aesgcm.encrypt(iv, pt, None)
    
    
    with open(file, 'wb') as h:
        h.write(salt + iv + ct)


def add_contact(phone_book, name, number):
    phone_book[name] = number
    print(f'Contact {name} added with number {number}.')


def search_contact(phone_book, query):
    hits = [(name, number) for name, number in phone_book.items() if name.find(query) != -1]

    if hits:
        print(f"Found {len(hits)} hits:")
        for name, number in hits:
            print(f"- {name}: {number}")
    else:
        print(f"No entries for query '{query}'")


def main():
    # Za vnos gesla uporabimo modul getpass
    password = getpass("Vpiši geslo: ")
    phone_book = load_phone_book(password, file_path)

    print(f"Found {len(phone_book)} contacts.")

    while True:
        print("\nPhone Book Menu:")
        print("1. Add Contact")
        print("2. Search Contact")
        print("3. Exit")

        choice = input("Enter your choice (1/2/3): ")

        match choice:
            case '1':
                name = input("Enter contact name: ")
                number = input("Enter contact number: ")
                add_contact(phone_book, name, number)
            case '2':
                name = input("Enter contact name to search: ")
                search_contact(phone_book, name)
            case '3':
                save_phone_book(phone_book, file_path, password)
                print("Phone book saved. Goodbye!")
                break
            case _:
                print("Invalid choice. Please enter 1, 2, or 3.")


if __name__ == "__main__":
    main()


## Fernet v Pythonu

Pomembni viri:
- [Implementacija Fernet v knjižnici cryptography](https://cryptography.io/en/latest/fernet/)
- [Specifikacija Fernet](https://github.com/fernet/spec/blob/master/Spec.md)
- [Python kodiranje z base64](https://docs.python.org/3/library/base64.html#base64.urlsafe_b64encode)

Kniznice
```java
import javax.crypto.SecretKeyFactory;
import javax.crypto.spec.PBEKeySpec;
import java.security.Key;
import java.security.SecureRandom;
import java.security.spec.KeySpec;
import java.util.HexFormat;
import java.nio.file.Files;
import java.nio.file.Path;
```

#### Read file

```java

public class ReadPhoneBookFile {
    public static void main(String[] args) throws Exception {
        // datoteko morate najprej ustvariti s programom pb.py
        final byte[] data = Files.readAllBytes(Path.of("pot_do_datoteke/phonebook.enc"));
    }
    // Definirajte pomožno metodo deriveKey in jo uporabite pri branju
    public static Key deriveKey(String password, byte[] salt, int iterations) throws Exception {
        return null;
    }
}
```

#### Fernet generiramo ključ

In [ ]:
k = Fernet.generate_key()   # generiramo ključ
f = Fernet(k)   # instaciiramo šifro Fernet
ct = f.encrypt("Dober večer, tole je primer sporočila šifriranega s šifro Fernet.".encode("utf8"))
pt = f.decrypt(ct)
print(f'Čistopis: {pt.decode("utf8")}')

def fernet_gen_key() -> bytes:
    key = Fernet.generate_key()
    return key

def fernet_decrypt(key: bytes, token: bytes) -> bytes:
    f = Fernet(key)
    pt = f.decrypt(token)
    return pt